# Grab the data

This notebook runs out to National Centers for Environmental Information and grabs the global summary of the year. It then filters it down to just the data we need to generate the contour plot in [this other notebook](hardiness.ipynb).

In [1]:
from io import BytesIO
import tarfile
from urllib.request import urlopen
import csv
import pandas as pd
import numpy as np

url = 'https://www.ncei.noaa.gov/data/gsoy/archive/gsoy-latest.tar.gz'
b = BytesIO(urlopen(url).read())


In [2]:

def extractor():
    with tarfile.open(mode='r', fileobj=b) as archive:
        for m in archive.getmembers():
            if not m.name.startswith('US'):
                continue

            yield archive.extractfile(m)
        

df_main = pd.DataFrame(columns=['DATE', 'LATITUDE', 'LONGITUDE', 'EMNT'])

gen = extractor()
for g in gen:
    df = pd.read_csv(g)
    if not 'EMNT' in df.keys():
        continue
    filtered = df[['DATE', 'LATITUDE', 'LONGITUDE', 'EMNT']].loc[df['DATE'] == 2019].dropna()
    df_main = pd.concat([df_main, filtered])
    

In [4]:
df_main.shape

(5874, 4)

In [5]:
df_main.head()

,DATE,LATITUDE,LONGITUDE,EMNT
46,2019,32.94520,-85.94800,-7.8
51,2019,33.12720,-88.15500,-11.1
15,2019,34.96285,-87.37195,-9.4
45,2019,34.77520,-86.95080,-8.3
56,2019,31.18200,-87.43900,-4.4


In [6]:
df_main.to_csv('gsoy_filtered.csv')